# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
cities_df=pd.read_csv("../data/cities.csv")
cities_df.head()

,City,Country,Wind Speed,Cloudiness,Max Temperature,Humidity,Lattitude,Longitude,Date
0,Mbamba,NG,6.64,100,70.86,93,9.9833,13.1000,1627301256
1,Torbay,CA,1.01,96,59.76,96,47.6666,-52.7314,1627301137
2,Pedra Branca,BR,10.40,6,76.68,59,-5.4542,-39.7172,1627301256
3,Mataura,NZ,7.72,100,48.47,94,-46.1927,168.8643,1627301256
4,Kruisfontein,ZA,19.42,100,73.89,21,-34.0033,24.7314,1627301257


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

#store coodinates
locations=cities_df[["Lattitude","Longitude"]]

#store humidity
humidity_df=cities_df["Humidity"]
humidity_df.astype(float)

0      93.0
1      96.0
2      59.0
3      94.0
4      21.0
       ... 
542    21.0
543    93.0
544    94.0
545    67.0
546    47.0
Name: Humidity, Length: 547, dtype: float64

In [7]:
fig=gmaps.figure(center=(46.0,-5.0),zoom_level=2)
max_humidity=np.max(humidity_df)

#heat layer
heat_layer=gmaps.heatmap_layer(locations,weights=humidity_df,dissipating=False,point_radius=3)

fig.add_layer(heat_layer)


plt.savefig('../Images/Humidity Heatmap.png')
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [8]:
personal_climate=cities_df.loc[(cities_df["Wind Speed"]<=20) & (cities_df["Cloudiness"]<=11)
                              &(cities_df["Humidity"]>=25) & (cities_df["Max Temperature"]<=95)
                              &(cities_df["Max Temperature"]>=60)].dropna()
personal_climate

,City,Country,Wind Speed,Cloudiness,Max Temperature,Humidity,Lattitude,Longitude,Date
2,Pedra Branca,BR,10.40,6,76.68,59,-5.4542,-39.7172,1627301256
22,Castro,BR,4.23,0,62.20,52,-24.7911,-50.0119,1627301263
28,Jamestown,US,1.01,1,70.30,98,42.0970,-79.2353,1627301264
32,Big Spring,US,5.75,1,74.68,66,32.2504,-101.4787,1627301265
35,Saint George,US,1.01,1,78.64,70,37.1041,-113.5841,1627301266
...,...,...,...,...,...,...,...,...,...
530,Cortez,US,4.61,1,60.69,94,37.3489,-108.5859,1627301423
533,Price,US,6.91,1,65.12,59,39.5994,-110.8107,1627301424
535,Naantali,FI,6.91,0,88.36,36,60.4674,22.0243,1627301424
539,Mumbwa,ZM,10.98,4,70.70,26,-14.9783,27.0619,1627301426


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df=personal_climate

hotel_df["Hotel Name"]=""

hotel_df=hotel_df[["City","Country","Lattitude","Longitude","Hotel Name"]]
hotel_df

,City,Country,Lattitude,Longitude,Hotel Name
2,Pedra Branca,BR,-5.4542,-39.7172,
22,Castro,BR,-24.7911,-50.0119,
28,Jamestown,US,42.0970,-79.2353,
32,Big Spring,US,32.2504,-101.4787,
35,Saint George,US,37.1041,-113.5841,
...,...,...,...,...,...
530,Cortez,US,37.3489,-108.5859,
533,Price,US,39.5994,-110.8107,
535,Naantali,FI,60.4674,22.0243,
539,Mumbwa,ZM,-14.9783,27.0619,


In [11]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params={"type":"hotel",
        "keyword":"hotel",
        "radius":5000,
         "key":g_key}

for index, row in hotel_df.iterrows():
    lat=row["Lattitude"]
    lng=row["Longitude"]
    city_name=row["City"]
    
    params["location"]=f"{lat},{lng}"
    
    print(f'Getting results for index {index}: {city_name}.')
    response=requests.get(url,params=params).json()
          
    results=response["results"]
          
    try:
          print(f"The Closest hotel in {city_name} is {results[0]['name']}.")
          hotel_df.loc[index,"Hotel Name"]=results[0]['name']
    except(KeyError,IndexError):
          print("Missing Information / Skipping")
          
    print("-----------")
          
print("END SEARCH")

Getting results for index 2: Pedra Branca.
The Closest hotel in Pedra Branca is Pousada Pedra Branca.
-----------
Getting results for index 22: Castro.


C:\Users\Nate\anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


The Closest hotel in Castro is Central Palace Hotel - Castro (PR).
-----------
Getting results for index 28: Jamestown.
The Closest hotel in Jamestown is Chautauqua Harbor Hotel.
-----------
Getting results for index 32: Big Spring.
The Closest hotel in Big Spring is Hotel Settles.
-----------
Getting results for index 35: Saint George.
The Closest hotel in Saint George is Quality Inn.
-----------
Getting results for index 48: Westport.
The Closest hotel in Westport is Residence Inn by Marriott Norwalk.
-----------
Getting results for index 51: Sangar.
Missing Information / Skipping
-----------
Getting results for index 53: Roald.
The Closest hotel in Roald is Smartrom.
-----------
Getting results for index 56: Albany.
Missing Information / Skipping
-----------
Getting results for index 57: Asău.
The Closest hotel in Asău is Pensiunea Sophia.
-----------
Getting results for index 61: Ponta do Sol.
The Closest hotel in Ponta do Sol is Palace of Zinos.
-----------
Getting results for ind

In [12]:
hotel_df.dropna(how='any')
hotel_df.to_csv('../data/Personal Climate Hotel List.csv')

In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lattitude", "Longitude"]]

In [14]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)


# Display figure
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))